# Code to replicate Figure 2 (b,c,d,f,g,h) and Figure 3.

# Installing packages

In [ ]:
# Required for tt decomposition to reduce the space complexity for storing tensors

!git clone --recursive https://github.com/oseledets/ttpy.git
!python ttpy/setup.py install --quiet


Cloning into 'ttpy'...
remote: Enumerating objects: 2128, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 2128 (delta 9), reused 17 (delta 5), pack-reused 2100 (from 1)
Receiving objects: 100% (2128/2128), 715.44 KiB | 9.41 MiB/s, done.
Resolving deltas: 100% (1381/1381), done.
Submodule 'tt/cross/rectcross' (https://bitbucket.org/oseledets/rectcross) registered for path 'tt/cross/rectcross'
Submodule 'tt/tt-fort' (https://github.com/oseledets/tt-fort.git) registered for path 'tt/tt-fort'
Submodule 'tt/utils/rect_maxvol' (https://bitbucket.org/muxas/rect_maxvol) registered for path 'tt/utils/rect_maxvol'
Cloning into '/content/ttpy/tt/cross/rectcross'...
remote: Enumerating objects: 112, done.        
remote: Counting objects: 100% (112/112), done.        
remote: Compressing objects: 100% (107/107), done.        
remote: Total 112 (delta 56), reused 0 (delta 0), pack-reused 0 (from 0)        
Receiving objects: 100% (1

# Importing packages

In [1]:
import numpy as np
import statistics
import random
from scipy.optimize import minimize
from scipy.linalg import expm
import time
import matplotlib.pyplot as plt
from scipy.stats import unitary_group
from sklearn.preprocessing import normalize
from itertools import combinations
import collections
import scipy
plt.style.use('ggplot')

In [2]:
# Tensor train packages

import tt
from tt.core.vector import vector

# Required functions

## Basic functions

In [3]:
# Initialize basic gates and operators

X_gate = np.matrix([[0 + 0j, 1 + 0j],
               [1 + 0j, 0+ 0j]])

Y_gate = np.matrix([[0 + 0j, -1j],
               [1j, 0 + 0j]])

Z_gate = np.matrix([[1 + 0j, 0 + 0j],
               [0 + 0j, -1 + 0j]])

ket_0_density = np.matrix([[1 + 0j, 0 + 0j],
                          [0 + 0j, 0 + 0j]])
swap = np.matrix([[1,0,0,0],
                  [0,0,1,0],
                  [0,1,0,0],
                  [0,0,0,1]])

SWAP = np.matrix([[1,0,0,0],
                 [0,0,1,0],
                 [0,1,0,0],
                 [0,0,0,1]])

SWAP_T = np.copy(SWAP)

big_std_projs = []

D = np.matrix([[2, 0],
              [0, -1]])

# This function returns the matrix form of a CNOT gate
# 1. is_rev: By default, the returned matrix assumes that that the first qubit is the control qubit. Set this to 1 to make second qubit control.
# 2. dtype: Data type of the entries of the returned matrix
def CNOT(is_rev=0, dtype='complex128'):
  if is_rev == 0:
    return np.matrix([[1 + 0j, 0 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 1 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 0 + 0j, 0 + 0j, 1 + 0j],
                  [0 + 0j, 0 + 0j, 1 + 0j, 0 + 0j]], dtype=dtype)

  else:
    return np.matrix([[1 + 0j, 0 + 0j, 0 + 0j, 0 + 0j],
                      [0 + 0j, 0 + 0j, 0 + 0j, 1 + 0j],
                      [0 + 0j, 0 + 0j, 1 + 0j, 0 + 0j],
                      [0 + 0j, 1 + 0j, 0 + 0j, 0 + 0j]], dtype=dtype)

# This function returns a numpy matrix of the form R_Z(theta) for any real number theta
def R_Z(theta, dtype = 'complex128'):
  return np.matrix([[np.exp(-1j * theta/2), 0],[0, np.exp(1j * theta/2)]], dtype=dtype)

# This function returns a numpy matrix of the form R_Y(theta) for any real number theta
def R_Y(theta, dtype = 'complex128'):
  return np.matrix([[np.cos(theta/2), (-1 * np.sin(theta/2))], [np.sin(theta/2), np.cos(theta/2)]], dtype=dtype)

# This function returns a numpy matrix of the form R_X(theta) for any real number theta
def R_X(theta, dtype = 'complex128'):
  return np.matrix([[np.cos(theta/2), ((-1j) * np.sin(theta/2))], [((-1j) * np.sin(theta/2)), np.cos(theta/2)]], dtype=dtype)

# Function to generate random 2 qubit clifford operator
def generate_random_clifford_circuit_qiskit():
  return np.matrix(random_clifford(2, seed=None).to_operator())

def create_big_std_projs(is32 = 0):
  big_std_projs = []
  for i in range(4):
    if is32 == 0:
      big_std_projs.append(np.outer(np.eye(4)[i], np.eye(4)[i]).astype('complex128'))

    else:
      big_std_projs.append(np.outer(np.eye(4)[i], np.eye(4)[i]).astype('float32'))
  return big_std_projs

paulis = [[np.eye(2), X_gate],
          [Z_gate, Y_gate]]

paulis_array = [[np.eye(2), np.array(X_gate)],
          [np.array(Z_gate), np.array(Y_gate)]]


# Pauli basis elements vectorized

P_basis = (1/(2 ** 0.5)) * np.matrix([[1,0,1,0],
                     [0,1,0,-1j],
                     [0,1,0,1j],
                     [1,0,-1,0]])

P_basis_T = P_basis.T
P_basis_cT = np.conj(P_basis).T

P_basis2 = np.kron(P_basis, P_basis)
P_basis2_T = np.conj(P_basis2).T

D_P = P_basis_cT @ np.array(D).flatten()

# Function to convert a number to binary
# x: number
# BitNo; number of bits in the binary expansion
def binary(x, BitNo):
  format(x, 'b').zfill(BitNo)
  Binlist = [int(y) for y in list(format(x, 'b').zfill(BitNo))]
  return Binlist

# Function to convert a number to any base expansion
# x: number
# bitNo; number of bits in the expansion
# base: base
def basary(x, bitno, base):
  ans_list = []
  q = np.copy(x)
  while q != 0:
    r = q % base
    q = q // base
    ans_list.append(r)
  ans_list = list(reversed(ans_list))
  if len(ans_list) < bitno:
    ans_list = [0] * (bitno - len(ans_list)) + ans_list
  return ans_list

# Functions to create pauli basis on n_qubits
def create_big_pauli_basis(n_qubits):
  mat = np.matrix([[1]])
  for i in range(n_qubits):
    mat = np.kron(mat, P_basis)

  return mat

# Function to compute the pauli basis entries of a density matrix
# state: density matrix as (2,2,..,,2) shape array
# max_index: the last index to compute
def pauli_basis_transformation_efficient(state, max_index):

  ans_list = []
  n_qubits = int(len(state.shape)/2)
  for i in range(max_index):
    temp_state = np.copy(state)
    if (i+1) % 1000 == 0:
      print(i+1)
    bin = binary(i, 2 * n_qubits)
    k = 0
    #print('i: ', i)
    for b in range(0, 2 * n_qubits, 2):
      pauli = paulis[bin[b]][bin[b+1]]
      #print(b)
      temp_state = np.tensordot(temp_state, np.conj(pauli), axes = [[0, n_qubits-k], [0,1]])
      k += 1
    #print(temp_state)
    ans_list.append(temp_state)
  return np.array(ans_list)

# Function that returns a random 2 qubit parameterized gate whose structure is given in the paper
def random_s_gate():
  return np.kron(unitary_group.rvs(2), unitary_group.rvs(2)) @ CNOT()

# Function that returns a random 2 qubit parameterized gate built using multiple layers whose structure is given in the paper
# depth: number of depth
def random_sd_gate(depth):
  mat = np.eye(4)
  for i in range(depth):
    mat = mat @ random_s_gate()
  return mat

# Function to apply a gate of a state
# state: an n-dimensional array representing a state or a list of core tensors representing the state given as MPS
# qubits_to_act: the qubits to act the gate on
# gate: gate
# mps: set it to 1 if the state is given as an mps
def apply_gate(state, qubits_to_act, gate, mps = 0):
  if mps == 0:
      temp_state = np.copy(state)
      temp_state = np.tensordot(temp_state, np.reshape(np.array(gate), tuple([temp_state.shape[0]] * (2 * len(qubits_to_act)))), axes = [qubits_to_act, list(range(-len(qubits_to_act),0))])
      temp_state = np.moveaxis(temp_state, list(range(-len(qubits_to_act),0)), qubits_to_act)
      return temp_state
  else:
      temp_state = list.copy(state)
      if len(qubits_to_act) == 1:
        temp_state = apply_single_qubit_gate(temp_state, gate, qubits_to_act)
      elif len(qubits_to_act) == 2:
        temp_state = apply_two_qubit_gate(temp_state, gate, qubits_to_act)
      else:
        temp_state = apply_multi_qubit_gate(temp_state, gate, qubits_to_act)
      return temp_state


def apply_matrix(state, qubits_to_act, gate):
      temp_state = np.copy(state)
      temp_state = np.tensordot(temp_state, gate, axes = [qubits_to_act, [0]])
      temp_state = np.moveaxis(temp_state, [-1], qubits_to_act)
      return temp_state

# Function to permute the indies of a density matrix tensor in such a way that enables pauli basis transformation
def permute_indices(tensor):
  n_qubits = int(len(tensor.shape)/2)
  indices = []
  for i in range(n_qubits):
    indices += [i,i+n_qubits]

  return np.ndarray.transpose(tensor, indices)

# Function to convert the gate in a manner that enables application on density given in pauli basis
def big_swaps(gate, big_pauli_basis_transformations = [], is32 = 0):
  gate_list = []
  n_qubits = int(np.log2(gate.shape[0])/2)
  gate_reshaped = np.copy(np.reshape(gate, tuple([2] * (4 * n_qubits))))
  indices = []
  for i in range(n_qubits):
    indices += [i, i + n_qubits]
  indices = indices + [x + (2 * n_qubits) for x in indices]
  gate_reshaped = np.ndarray.transpose(gate_reshaped, indices)

  if len(big_pauli_basis_transformations) == 0:
    return np.reshape(gate_reshaped, (4 ** n_qubits, 4 ** n_qubits))

  else:
    if is32 == 0:
      return np.real(big_pauli_basis_transformations[1] @ np.reshape(gate_reshaped, (4 ** n_qubits, 4 ** n_qubits)) @ big_pauli_basis_transformations[0])

    else:
      ans = np.real(big_pauli_basis_transformations[1] @ np.reshape(gate_reshaped, (4 ** n_qubits, 4 ** n_qubits)) @ big_pauli_basis_transformations[0])
      return ans.astype('float32')


# Function to create a subcircuit for the MPS ansatz. The subcircuit has the form e^{-i\sum \limits_j \theta_j P)j} where \theta is a vector of real parameters and P_js are paulis
# theta: vector of parameters
# sampled_paulis: list of Paulis
def step_circuit(theta, sampled_paulis):
  mat = sum([theta[i] * sampled_paulis[i] for i in range(len(theta))])
  gate = expm(1j * mat)
  return gate


## MPS related

In [4]:
pb = create_big_pauli_basis(2)
big_pauli_transformation = [pb, np.conj(pb).T]
SWAP_P = big_swaps(np.array(np.kron(SWAP, SWAP)), big_pauli_transformation)



# Function to apply a single qubit gate to an mps
# mps: mps given as a list of core tensors
# gate: gate
# qubit: qubit index
def apply_single_qubit_gate(mps, gate, qubit):
  new_mps = list.copy(mps)
  temp = np.tensordot(gate, new_mps[qubit], axes=([1], [0]))
  new_mps[qubit] = np.copy(temp)
  return new_mps

# Function to apply a two qubit gate to an mps
# mps: mps given as a list of core tensors
# gate: gate
# qubits: list of two qubit indices
def apply_two_qubit_gate(mps, gate, qubits):
  dim = mps[qubits[0]].shape[0]
  if np.abs(qubits[0] - qubits[1]) == 1:

    chi1 = mps[qubits[0]].shape[1]
    chi2 = mps[qubits[1]].shape[2]
    gate = np.reshape(np.array(gate), (dim,dim,dim,dim))
    temp = np.tensordot(gate, mps[qubits[0]], axes=([2], [0]))
    temp = np.tensordot(temp, mps[qubits[1]], axes=([2, 4], [0, 1]))
    temp = np.ndarray.transpose(temp, [0,2,1,3])
    temp = np.reshape(temp, (dim * chi1, dim * chi2))
    u, sigma, vh = np.linalg.svd(temp, full_matrices = False)
    u = u @ np.diag(sigma)
    u = np.reshape(u, (dim, chi1, len(sigma)))
    vh = np.reshape(vh, (len(sigma), dim, chi2))
    vh = np.ndarray.transpose(vh, [1,0,2])
    mps[qubits[0]] = u
    mps[qubits[1]] = vh
    return mps
  else:
    l = min(qubits)
    m = max(qubits)
    temp_mps = list.copy(mps)
    for i in range(l, m-1):
      #print("Swapping ", [i, i+1])
      temp_mps = apply_two_qubit_gate(temp_mps, SWAP_P, [i, i+1])
    #print('m', m)
    if qubits[0] < qubits[1]:
      #print("Applying ", [m-1, m])
      temp_mps = apply_two_qubit_gate(temp_mps, gate, [m-1, m])

    for i in range(m-2, l-1, -1):
      #print("Swapping ", [i, i+1])
      temp_mps = apply_two_qubit_gate(temp_mps, SWAP_P, [i, i+1])
    return temp_mps

# Function to divide an MPS by a scalar
def div_by_scalar(mps, scalar):
  ans = []
  for i in range(len(mps)):
    ans.append(mps[i]/(scalar ** (1/len(mps))))

  return ans

# Function to apply the inverse of a QCNN on an MPS
# state: mps given as a list of core tensors
# theta: an array of parameters of the QCNN
# sampled_paulis: list of paulis that prepares a parameterized gate of the form e^{-i\sum \limits_j \theta_j P)j} where \theta is a vector of real parameters and P_js are paulis in this list
# big_pauli_basis_transformation: pauli basis transformation matrix defined on 2 qubits, already initialized previously
# is_random: set to 1 if the 2 qubit subcircuits are haar random, 2 if the two qubit subcircuits are from the list "random_gates", anything else to use the paraemters taken as theta
# random_gates: set of 2 qubit gates
def qcnn_inverse_mps(state, theta, sampled_paulis = 0, big_pauli_basis_transformations = [], is_random = 0, random_gates = 0):
  temp_state = list.copy(state)
  n_qubits = len(state)
  pos = 0
  len_s = 4 ** 2
  r = 0
  for d in range(int(np.log2(n_qubits))-1, -1, -1):
    #print(d)
    #for i in range((2 ** d)-1, n_qubits, 2 ** (d+1)):
    for i in range(n_qubits-1-(2 ** d), (2 ** d)-2, -(2 ** (d+1))):
      #print([i, i+(2 ** d)])
      #print(is_random)
      if is_random == 1:
        tq_gate = unitary_group.rvs(4)

      elif is_random == 2:
        tq_gate = random_gates[r]
        r += 1
      else:
        tq_gate = np.conj(step_circuit(theta[len_s * pos: len_s * (pos + 1)], sampled_paulis)).T
      gate = big_swaps(np.kron(tq_gate, np.conj(tq_gate)), big_pauli_basis_transformations)
      temp_state = apply_two_qubit_gate(temp_state, gate, [i, i+(2 ** d)])
      temp_state = round_mps(temp_state, 0.000000001)
      pos += 1
  return temp_state

# Function to apply an HEA on a state given as a tensor or an MPS
# theta: an array of parameters of the HEA
# state: state given as an n-dimensional tensor or mps given as a list of core tensors
# depth: depth of the HEA
# sampled_paulis: list of paulis that prepares a parameterized gate of the form e^{-i\sum \limits_j \theta_j P)j} where \theta is a vector of real parameters and P_js are paulis in this list
# mps: 0 if the state is given as a tensor, 1 if the state is given as an MPS
# big_pauli_basis_transformation: pauli basis transformation matrix defined on 2 qubits, already initialized previously
# is_random: set to 1 if the 2 qubit subcircuits are random_sd_gates, 2 if the two qubit subcircuits are from the list "random_gates", 0 to use the paraemters taken as theta
# random_gates: set of 2 qubit gates
def ala(theta, state, depth, sampled_paulis = 0, mps = 0, big_pauli_basis_transformations = [], is_random = 0, random_gates = 0):
  if mps == 0:
    temp_state = np.copy(state)
    n_qubits = len(state.shape)
  else:
    temp_state = list.copy(state)
    n_qubits = len(state)
  #depth = int(np.log2(n_qubits))
  pos = 0
  r = 0
  len_s = 4 ** 2
  for d in range(depth):
    for i in range(0, n_qubits, 2):
      if ((i + 1 + (d % 2)) % n_qubits) != 0:
        if is_random == 0:
          tq_gate = step_circuit(theta[len_s * pos: len_s * (pos + 1)], sampled_paulis)
        elif is_random == 2:
          tq_gate = random_gates[r]
          r += 1
        else:
          tq_gate = np.array(random_sd_gate(1))
        #print(qubits[i + (d % 2)], qubits[(i + 1 + (d % 2)) % n_qubits])
        if mps == 0:
          gate = big_swaps(np.kron(tq_gate, np.conj(tq_gate)), big_pauli_basis_transformations)
          temp_state = apply_gate(temp_state, [i + (d % 2), (i + 1 + (d % 2)) % n_qubits], gate)
        else:
          gate = big_swaps(np.kron(tq_gate, np.conj(tq_gate)), big_pauli_basis_transformations)
          #print([i + (d % 2), (i + 1 + (d % 2)) % n_qubits])
          temp_state = apply_two_qubit_gate(temp_state, gate, [i + (d % 2), (i + 1 + (d % 2)) % n_qubits])
          temp_state = round_mps(temp_state, 0.000000001)
        pos += 1
  return temp_state

# Function to apply an mps_ansatz on a state given as an MPS
# temp_mat: mps given as a list of core tensors
# theta: an array of parameters of the MPS ansatz
# step_size: width of the subcircuit
# big_pauli_basis_transformation: pauli basis transformation matrix defined on 2 qubits, already initialized previously
# is_random: set to 2 if the two qubit subcircuits are from the list "random_circuits", anything else to use the paraemters taken as theta
# random_circuits: set of 2 qubit gates
def mps_ansatz(temp_mat, theta, step_size, big_pauli_transformation, is_random = 0, random_circuits = []):
  mat = list.copy(temp_mat)
  n_qubits = len(mat)

  gate_list = []
  #step_size = int(np.log2(n_qubits))
  r = 0
  for i in range(n_qubits - step_size + 1):
    for d in range(step_size):
      for j in range(i + (d%2), i + step_size, 2):
        if (j + 1) < (i + step_size):
          if is_random == 2:
            tq_gate = random_circuits[r]
            #tq_gate = unitary_group.rvs(4)
            r += 1
          else:
            tq_gate = np.array(random_s_gate())
          gate_list.append((tq_gate, [j, j+1]))

    # for j in range(i, i + step_size):
    #   if (j + 1) < (i + step_size):
    #     gate_list.append((np.array(random_s_gate()), [j, j+1]))

  for i in range(len(gate_list)-1, -1, -1):
  #for i in range(len(gate_list)):
    #print(gate_list[i][1])
    tq_gate = big_swaps(np.kron(gate_list[i][0], np.conj(gate_list[i][0])), big_pauli_transformation)
    #mat = apply_gate(mat, gate_list[i][1], tq_gate)
    mat = apply_two_qubit_gate(mat, np.array(tq_gate), gate_list[i][1])
    mat = round_mps(mat, 0.000000000001)
  return mat

# Function to creates the MPS decomposition Z_n in Pauli basis
def create_z_pauli_mps(n_qubits):
  mat = []
  for i in range(n_qubits):
    if i == n_qubits-1:
      mat += [np.reshape(np.array([0, 0, 2 ** 0.5, 0]), (4,1,1))]
      #mat = np.kron(mat, np.array([0, 0, 2 ** 0.5, 0]))
    else:
      mat += [np.reshape(np.array([2 ** 0.5, 0, 0, 0]), (4,1,1))]
      #mat = np.kron(mat, np.array([2 ** 0.5, 0, 0, 0]))

  return mat

# Function to creates the MPS decomposition of \ket{0}\bra{0}_n in Pauli basis
def create_ket0_pauli_mps(n_qubits):
  mat = []
  for i in range(n_qubits):
    if i == n_qubits-1:
      mat += [np.reshape(np.array([1/(2 ** 0.5), 0, 1/(2 ** 0.5), 0]), (4,1,1))]
      #mat = np.kron(mat, np.array([0, 0, 2 ** 0.5, 0]))
    else:
      mat += [np.reshape(np.array([2 ** 0.5, 0, 0, 0]), (4,1,1))]
      #mat = np.kron(mat, np.array([2 ** 0.5, 0, 0, 0]))

  return mat

# Function to creates the MPS decomposition of \ket{0}\bra{0}_k in Pauli basis
def create_ket0_pauli_mps_i(n_qubits, k):
  mat = []
  for i in range(n_qubits):
    if i == k:
      mat += [np.reshape(np.array([1/(2 ** 0.5), 0, 1/(2 ** 0.5), 0]), (4,1,1))]
      #mat = np.kron(mat, np.array([0, 0, 2 ** 0.5, 0]))
    else:
      mat += [np.reshape(np.array([2 ** 0.5, 0, 0, 0]), (4,1,1))]
      #mat = np.kron(mat, np.array([2 ** 0.5, 0, 0, 0]))

  return mat

# Function to create the MPS decomposition of \sum \limits_{k=1}^n \ket{0}\bra{0}_k in Pauli basis
def create_all0_pauli_mps_sum(n_qubits):
  mps = create_ket0_pauli_mps_i(n_qubits, 0)
  for i in range(1, n_qubits):
    mps = add_mps_vector(mps, create_ket0_pauli_mps_i(n_qubits, i))
    mps = round_mps(mps, 0.000000000001)
  return mps

# Function that creates the MPS decomposition of all0 state in pauli basis
def create_all0_pauli_mps(n_qubits):
  return [np.reshape(np.array([1/(2 ** 0.5), 0, 1/(2 ** 0.5), 0]), (4,1,1))] * n_qubits

# Function that creates the MPS deocmposition of Z \otimes \otimes Z \otimes \dots \otimes Z in pauli basis
def create_allz_pauli_mps(n_qubits):
  return [np.reshape(np.array([0, 0, 2 ** 0.5, 0]), (4,1,1))] * n_qubits

def create_random_s(depth = 1):
  theta = np.random.uniform(0, 2 * np.pi, 2 * depth)
  return S_gate(theta)

# Function to query the element in an index of an MPS
def query_mps(mps, index):
    n_qubits = len(mps)
    ans_list = []
    #q = np.copy(index)
    #ans = np.matrix([[1]])
    dim = mps[0].shape[0]
    #bin_list = binary(index, n_qubits)
    if type(index) != list:
      bin_list = basary(index, n_qubits, dim)

    else:
      bin_list = list.copy(index)
    for div in range(n_qubits):
        if div == 0:
          ans = mps[div][bin_list[div]]
        else:
          ans = ans @ mps[div][bin_list[div]]
    return ans[0,0]
    #return ans

# Function that computes the inner product between two MPSs
def inner_product(mps1, mps2):

    for i in range(len(mps1)):
        for j in range(mps1[i].shape[0]):
            temp = np.kron(mps1[i][j,:,:], np.conj(mps2[i][j,:,:]))
            if j == 0:
                A = temp
            else:
                A += temp

        if i == 0:
            ans_mat = A

        else:
            ans_mat = ans_mat @ A
    #print(ans_mat)
    return np.trace(ans_mat)

# Function to measure a single qubit of an MPS
def measure_single_qubit_mps(mps, qubit):
    states = []
    probs = []
    #print("Inner product: ", inner_product(mps, mps))
    for i in range(4):
      proj_mps = apply_single_qubit_gate(mps, big_std_projs[i], qubit)
      p0 = inner_product(proj_mps, mps)
      states.append(proj_mps)
      probs.append(p0)
      #proj_mps = apply_single_qubit_gate(mps, np.eye(2), qubit)

    #print("Sum of probs: ", sum(probs))
    ans = np.random.choice(list(range(4)), size = 1, p = probs)

    return div_by_scalar(states[ans[0]], probs[ans[0]] ** 0.5), ans[0]

# Function to compute the expectation of the MPS with O
def measure_individual_qubits_mps(mps):
  temp_mps = list.copy(mps)
  n_qubits = len(mps)
  ans = 0
  for i in range(n_qubits):
     #print('i: ', i)
     ans += inner_product(round_mps(apply_single_qubit_gate(temp_mps, ket_0, i), 0.000000001), mps)/n_qubits
  return ans

# Function that computes the P-norm of a matrix given as MPS
def norm_4_mps_efficient(mps):
  mps1 = list.copy(mps)
  for i in range(len(mps1)):
      #print(i)
      if i == 0:
        for j in range(mps1[i].shape[0]):
            temp = np.kron(mps1[i][j,:,:], np.conj(mps1[i][j,:,:]))
            temp = np.kron(temp, temp)
            if j == 0:
                A = temp
            else:
                A += temp


        ans_mat = A
        ans_mat = ans_mat.flatten()
        size = int(len(ans_mat) ** (1/4))
        ans_mat = np.reshape(ans_mat, (size, size, size, size))

        #print(ans_mat.flatten().shape)

      else:
        for j in range(mps1[i].shape[0]):
            #print('j', j)
            temp = np.copy(mps1[i][j,:,:])
            temp_ans_mat = np.copy(ans_mat)
            for k in range(4):
                temp_ans_mat = apply_matrix(temp_ans_mat, [k], temp)
            if j == 0:
                new_ans_mat = temp_ans_mat

            else:
                new_ans_mat = new_ans_mat + temp_ans_mat

        ans_mat = np.copy(new_ans_mat)
        #return

  return ans_mat

# Function that computes the direct sum of two matrices
def block_mps(a, b):
  return np.block([[a, np.zeros((a.shape[0], b.shape[1]))],
            [np.zeros((b.shape[0], a.shape[1])), b]])

# Function to compute the expectation of pauli strings involving I and Z only.
# mps: input mps
# index: a list indicating the presence of z in the pauli string; [0,0,1] is I \otimes I \otimes Z
def query_z_mps(mps, index):
    n_qubits = len(mps)
    ans_list = []
    #q = np.copy(index)
    #ans = np.matrix([[1]])
    #dim = mps[0].shape[0]
    #bin_list = binary(index, n_qubits)
    if type(index) != list:
      #bin_list = basary(index, n_qubits, dim)
      bin_list = binary(index, n_qubits)

    else:
      bin_list = list.copy(index)

    new_bin_list = []
    for div in range(n_qubits):
      if bin_list[div] == 0:
        new_bin_list.append(0)
      else:
        new_bin_list.append(2)

    #new_bin_list =list(reversed(new_bin_list))
    for div in range(n_qubits):
        if div == 0:
          ans = mps[div][new_bin_list[div]]
        else:
          ans = ans @ mps[div][new_bin_list[div]]
    return ans[0,0]


# Function that adds two MPSs
def add_mps_vector(temp_mps1, temp_mps2):
  ans_mps = []
  mps1 = list.copy(temp_mps1)
  mps2 = list.copy(temp_mps2)

  for i in range(len(mps1)):
    if i == 0:
      ans_mps.append(np.array([np.hstack([mps1[i][0,:,:], mps2[i][0,:,:]]),
                              np.hstack([mps1[i][1,:,:], mps2[i][1,:,:]]),
                              np.hstack([mps1[i][2,:,:], mps2[i][2,:,:]]),
                              np.hstack([mps1[i][3,:,:], mps2[i][3,:,:]])]))

    elif i == n_qubits-1:
      ans_mps.append(np.array([np.vstack([mps1[i][0,:,:], mps2[i][0,:,:]]),
                              np.vstack([mps1[i][1,:,:], mps2[i][1,:,:]]),
                              np.vstack([mps1[i][2,:,:], mps2[i][2,:,:]]),
                              np.vstack([mps1[i][3,:,:], mps2[i][3,:,:]])]))


    else:
      ans_mps.append(np.array([block_mps(mps1[i][0,:,:], mps2[i][0,:,:]),
                             block_mps(mps1[i][1,:,:], mps2[i][1,:,:]),
                             block_mps(mps1[i][2,:,:], mps2[i][2,:,:]),
                             block_mps(mps1[i][3,:,:], mps2[i][3,:,:])]))
  return ans_mps

# Function to divide an MPS by a scaler
def div_by_scalar(mps, scalar):
  ans = []
  for i in range(len(mps)):
    ans.append(mps[i]/(scalar ** (1/len(mps))))

  return ans

# Function that generates the distribution of the Pauli basis coefficients of observables conjugated by MPS ansatz
# obs: observable given as MPS
# final_index: final index of the Pauli basis coefficients
def ce_distribution(obs, final_index, interval = 1000):
  distribution = []
  n_qubits = len(obs)
  for i in range(final_index):
    if (i + 1) % interval == 0:
      print(i + 1)
    distribution.append(query_z_mps(obs, i) ** 2)
  return distribution

#  ala(theta, state, depth, sampled_paulis = 0, mps = 0, big_pauli_basis_transformations = [], is_random = 0, random_gates = 0)


## Functions required for TT decomposition

In [5]:
def change_tensor_order(mps):
    m = []
    for i in range(len(mps)):
        m.append(np.ndarray.transpose(mps[i], [1, 0, 2]))

    return m

def round_mps(mps, eps):
  v = vector()
  a = v.from_list(change_tensor_order(mps))
  ans_mps = change_tensor_order(v.to_list(a.round(eps)))
  return ans_mps


# Comparison of C-E norms with second moments of the cost function

## MPS Ansatz

In [6]:
# Function that estimates the C-E norms and squared cost functions of a single random input states genreated using HEA
# n_qubits: number of qubits
# step_size: width of the subcircuit used
# big_pauli_transformation: initialized earlier
def generate_random_var_4_norm(n_qubits, step_size, big_pauli_transformation):
  depth = int(np.log2(n_qubits))
  rho_theta = np.random.uniform(0, 2 * np.pi, 10000)
  fro = (((2 ** (n_qubits/2))/n_qubits) + (((n_qubits-1) * (2 ** (n_qubits/4)))/n_qubits)) ** 0.5
  #fro = 2 ** (n_qubits/2)
  print('Preparing States')
  rho = create_all0_pauli_mps(n_qubits)
  rho = ala(rho_theta, rho, depth, mps = 1, big_pauli_basis_transformations = big_pauli_transformation, is_random = 1)
  obs_theta = np.random.uniform(0, 2 * np.pi, 10000)
  print('Preparing Observables')
  #obs_z = create_z_pauli_mps(n_qubits)
  #obs_z = div_by_scalar(create_all0_pauli_mps_sum(n_qubits), n_qubits)
  obs_z = create_ket0_pauli_mps(n_qubits)
  obs_allz = create_all0_pauli_mps(n_qubits)
  obs_sum_z = div_by_scalar(create_all0_pauli_mps_sum(n_qubits), n_qubits)

  #obs_z = div_by_scalar(obs_z, fro ** 0.5)
  obs_z_theta = mps_ansatz(obs_z, obs_theta, step_size, big_pauli_transformation, is_random = 1, random_circuits = 0)
  obs_allz_theta = mps_ansatz(obs_allz, obs_theta, step_size, big_pauli_transformation, is_random = 1, random_circuits = 0)
  obs_sumz_theta = mps_ansatz(obs_sum_z, obs_theta, step_size, big_pauli_transformation, is_random = 1, random_circuits = 0)
  print('Computing Inner Product')
  ip_z = inner_product(rho, obs_sumz_theta) ** 2
  ip_allz = inner_product(rho, obs_allz_theta) ** 2
  print('Computing Norms')
  #norm_z = (norm_4_mps_efficient(obs_z_theta) ** (1/4))
  #fro = 2 ** (n_qubits/2)
  fro = (2 ** n_qubits)/2
  norm_z = (norm_4_mps_efficient(obs_z_theta).flatten()[0] ** (1/4))/(fro ** 0.5)
  norm_allz = norm_4_mps_efficient(obs_allz_theta).flatten()[0] ** (1/4)

  return ip_z, ip_allz, norm_z, norm_allz


In [7]:
step_circuit_len = 2 # this is number of qubits of subcircuits being used within the subcircuits. Its always 2.
n_qubits = 30
pb = create_big_pauli_basis(step_circuit_len) # Create a pauli basis transformation for the aforementioned 2 qubit subcircuits
big_pauli_transformation = [pb, np.conj(pb).T]

step_size = int(np.log2(n_qubits))


In [8]:
ans = generate_random_var_4_norm(n_qubits, step_size, big_pauli_transformation)

## QCNN

In [ ]:
# Function that estimates the C-E norms and squared cost functions of a single random input states genreated using HEA
# n_qubits: number of qubits
# big_pauli_transformation: initialized earlier
def generate_random_var_4_norm(n_qubits, big_pauli_transformation):
  depth = int(np.log2(n_qubits))
  rho_theta = np.random.uniform(0, 2 * np.pi, 10000)
  print('Preparing States')
  rho = create_all0_pauli_mps(n_qubits)
  rho = ala(rho_theta, rho, depth, mps = 1, big_pauli_basis_transformations = big_pauli_transformation, is_random = 1)
  obs_theta = np.random.uniform(0, 2 * np.pi, 10000)
  print('Preparing Observables')
  obs_z = create_z_pauli_mps(n_qubits)
  obs_allz = create_allz_pauli_mps(n_qubits)
  obs_z_theta = qcnn_inverse_mps(obs_z, obs_theta, 0, big_pauli_transformation, is_random = 1)
  obs_allz_theta = qcnn_inverse_mps(obs_allz, obs_theta, 0, big_pauli_transformation, is_random = 1)
  print('Computing Inner Product')
  ip_z = inner_product(rho, obs_z_theta) ** 2
  ip_allz = inner_product(rho, obs_allz_theta) ** 2
  print('Computing Norms')
  norm_z = (norm_4_mps_efficient(obs_z_theta) ** (1/4))/(2 ** (n_qubits/2))
  norm_allz = (norm_4_mps_efficient(obs_allz_theta) ** (1/4))/(2 ** (n_qubits/2))

  return ip_z, ip_allz, norm_z, norm_allz


In [ ]:
step_circuit_len = 2 # this is number of qubits of subcircuits being used within the subcircuits. Its always 2.
n_qubits = 16
pb = create_big_pauli_basis(step_circuit_len) # Create a pauli basis transformation for the aforementioned 2 qubit subcircuits
big_pauli_transformation = [pb, np.conj(pb).T]

In [ ]:
ans = generate_random_var_4_norm(n_qubits, big_pauli_transformation)


Preparing States
Preparing Observables
Computing Inner Product
Computing Norms


In [ ]:
ans

(0.030118386679060728,
 1.9508612151752303e-05,
 array([[[[0.18469981]]]]),
 array([[[[0.01893453]]]]))

## HEA

In [ ]:
# Function that estimates the C-E norms and squared cost functions of a single random input states genreated using HEA
# n_qubits: number of qubits
# big_pauli_transformation: initialized earlier
def generate_random_var_4_norm(n_qubits, big_pauli_transformation):
  depth = int(np.log2(n_qubits))
  rho_theta = np.random.uniform(0, 2 * np.pi, 10000)
  print('Preparing States')
  rho = create_all0_pauli_mps(n_qubits)
  rho = ala(rho_theta, rho, depth, mps = 1, big_pauli_basis_transformations = big_pauli_transformation, is_random = 1)
  obs_theta = np.random.uniform(0, 2 * np.pi, 10000)
  print('Preparing Observables')
  obs_z = create_z_pauli_mps(n_qubits)
  obs_allz = create_allz_pauli_mps(n_qubits)
  obs_z_theta = ala(obs_theta, obs_z, int(np.log2(n_qubits)), sampled_paulis = 0, mps = 1, big_pauli_basis_transformations = big_pauli_transformation, is_random = 1, random_gates = 0)
  obs_allz_theta = ala(obs_theta, obs_allz, int(np.log2(n_qubits)), sampled_paulis = 0, mps = 1, big_pauli_basis_transformations = big_pauli_transformation, is_random = 1, random_gates = 0)
  print('Computing Inner Product')
  ip_z = inner_product(rho, obs_z_theta) ** 2
  ip_allz = inner_product(rho, obs_allz_theta) ** 2
  print('Computing Norms')
  norm_z = (norm_4_mps_efficient(obs_z_theta) ** (1/4))/(2 ** (n_qubits/2))
  norm_allz = (norm_4_mps_efficient(obs_allz_theta) ** (1/4))/(2 ** (n_qubits/2))

  return ip_z, ip_allz, norm_z, norm_allz


In [ ]:
step_circuit_len = 2 # this is number of qubits of subcircuits being used within the subcircuits. Its always 2.
n_qubits = 32
pb = create_big_pauli_basis(step_circuit_len) # Create a pauli basis transformation for the aforementioned 2 qubit subcircuits
big_pauli_transformation = [pb, np.conj(pb).T]

In [ ]:
ans = generate_random_var_4_norm(n_qubits, big_pauli_transformation)


Preparing States
Preparing Observables
Computing Inner Product
Computing Norms


In [ ]:
ans

(0.026886670709669616,
 1.9414565495918085e-09,
 array([[[[0.33593035]]]]),
 array([[[[0.00016534]]]]))

# Generating the Pauli basis distributions of observables conjugated by MPS ansatzes

In [12]:
n_qubits = 16
theta = 0 # not required. hence set it to 0
step_circuit_len = 2 # this is number of qubits of subcircuits being used within the subcircuits. Its always 2.
step_size = 4 # width of the subcircuit within the MPS ansatz
pb = create_big_pauli_basis(step_circuit_len) # Create a pauli basis transformation for the aforementioned 2 qubit subcircuits
big_pauli_transformation = [pb, np.conj(pb).T]
total_samples = 10 # number of samples required within the box plots

# Create a set of random 2 qubit circuits
random_circuits = []

for i in range(total_samples):
  temp = []
  for j in range(2000):
    temp.append(np.array(random_sd_gate(1)))
  random_circuits.append(temp)

ans = []
for i in range(total_samples):
  #mps = create_all0_pauli_mps(n_qubits)
  mps = create_ket0_pauli_mps(n_qubits)
  mps = mps_ansatz(mps, theta, step_size, big_pauli_transformation, is_random = 2, random_circuits = random_circuits[i])
  ans.append(ce_distribution(mps, 65536, interval = 50000))

50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
